# Mark Post processing


In [ ]:
%pip install pandas openpyxl

In [ ]:
import pandas as pd

df = pd.read_csv('marks.csv')
df.head()

In [ ]:
df = df.loc[:, ~df.columns.str.endswith('@type')]
df.head()

In [ ]:
df.dropna(subset=['passAt'], inplace=True)

In [ ]:
df['firstTrial'] = pd.to_datetime(df['firstTrial'], format="ISO8601")
df['lastTrial'] = pd.to_datetime(df['lastTrial'], format="ISO8601")
df['passAt'] = pd.to_datetime(df['passAt'], format="ISO8601")
df['PartitionKey'] = df['PartitionKey'].astype(str)
df['RowKey'] = df['RowKey'].astype(str)

In [ ]:
df.dtypes

In [ ]:
df['RowKey'] = df['RowKey'].str.replace('AzureProjectTestLib.', '')


In [ ]:
df.head()

In [ ]:
pass_at_df = pd.pivot_table(df, values='passAt', index=['PartitionKey'], columns=['RowKey'])
print(pass_at_df)


In [ ]:
# convert pass_at_df each column to ranking where the earliest time is 1, the second earliest is 2 .. set NaT to 0. Skip the first column
pass_at_df = pass_at_df.apply(lambda x: pd.Series(pd.factorize(x, sort=True)[0]+1, index=x.index) if x.dtype.kind == 'M' else x)
pass_at_df.iloc[:, 1:] = pass_at_df.iloc[:, 1:].astype(int)
print(pass_at_df)


In [ ]:
pass_at_df = pass_at_df.apply(lambda x: x.dt.tz_localize(None) if x.dtype.kind == 'M' else x)
pass_at_df.to_excel('pass_at.xlsx')
